### Printing out the Branches in the data.

In [ ]:
import ROOT
f = ROOT.TFile.Open("/mnt/data/eos/run307/run307.root")
events = f.Get("events")
events.Print()

## SET THE RUN NUMBER!

In [ ]:
run_number = "312"

### PMT Hits Map ###
The code block will open the root data file and grab the branches (some of these are vectors).<br>
Then loop through each of the event and hit and make a histogram out of the number of hits for each PMT.<br>
Then the map of Board# vs Channel are made.<br>
Board 14 Channel 12 to Board 15 Channel 14 are those will Muon paddle outputs.<br>

In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pandas as pd

rdf = ROOT.RDataFrame("events","/mnt/data/eos/run"+run_number+"/run"+run_number+".root")
nhit = rdf.AsNumpy(columns=["nhit"])["nhit"]
event_number = rdf.AsNumpy(columns=["event_number"])["event_number"]
board = rdf.AsNumpy(columns=["board"])["board"]
channel = rdf.AsNumpy(columns=["channel"])["channel"]
charge = rdf.AsNumpy(columns=["charge"])["charge"]
deltat = rdf.AsNumpy(columns=["deltat"])["deltat"]

#print(len(charge[0]))
#print(nhit[0])

PMTHits = np.zeros([17,16])
#print(PMTHits.shape)
for iEvt in range(0,len(event_number)):
    #print(len(board[aEvt]),len(channel[aEvt]), nhit[aEvt])
    for iHit in range(0,len(board[iEvt])):
        #print(board[iEvt][iHit])
        PMTHits[board[iEvt][iHit],channel[iEvt][iHit]]=PMTHits[board[iEvt][iHit],channel[iEvt][iHit]]+1

plt.figure(figsize=(17/2,16/2))
plt.imshow(PMTHits, cmap='jet',norm=colors.LogNorm())
plt.title("Total Number of Hits in Run "+run_number+" for each CAEN Board vs Channel")
plt.xlabel("Channel #")
plt.ylabel("Board #")
plt.colorbar()
plt.show()

nMuonHits = np.sum(PMTHits[14:16,11:15])
nTotalHits= np.sum(PMTHits)
print("nMuonHits nTotalHits nMuonHits/nTotalHits")
print(nMuonHits,nTotalHits,nMuonHits/nTotalHits)
print("nhits in B14C11-B15-C14")
print(PMTHits[14:16,11:15])


### Now, let's see if we can get coincidences of the paddles.
The code opens the file, grab the branches<br>
Then it loops through each event.<br>
    Create a 2x3 array (for the 6 powered up muon paddles) for each event.<br>
    Then fill the array with whether or not a hit occurred in the paddles.<br>
    Then it looks in the array to see if there are coincidences between top, middle, bottom paddles<br>

In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

run_number = "357"

rdf = ROOT.RDataFrame("events","/mnt/data/eos/run"+run_number+"/run"+run_number+".root")
nhit = rdf.AsNumpy(columns=["nhit"])["nhit"]
event_number = rdf.AsNumpy(columns=["event_number"])["event_number"]
board = rdf.AsNumpy(columns=["board"])["board"]
channel = rdf.AsNumpy(columns=["channel"])["channel"]
charge = rdf.AsNumpy(columns=["charge"])["charge"]

nZero=0
nSingle=0
nDouble=0
nTriple=0

nhitZero = []
nhitSingle = []
nhitDouble = []
nhitTriple = []


for iEvt in range(0,len(event_number)):
    MuonHits = np.zeros([2,3])    
    for iHit in range(0,len(board[iEvt])):
        if  (board[iEvt][iHit] == 14 and channel[iEvt][iHit] == 12):
           MuonHits[0,0]=1
        elif(board[iEvt][iHit] == 14 and channel[iEvt][iHit] == 13):
           MuonHits[0,1]=1      
        elif(board[iEvt][iHit] == 14 and channel[iEvt][iHit] == 14):
           MuonHits[0,2]=1
        elif(board[iEvt][iHit] == 15 and channel[iEvt][iHit] == 12):
           MuonHits[1,0]=1
        elif(board[iEvt][iHit] == 15 and channel[iEvt][iHit] == 13):
           MuonHits[1,1]=1
        elif(board[iEvt][iHit] == 15 and channel[iEvt][iHit] == 14):
           MuonHits[1,2]=1

    if(np.sum(MuonHits)==0): 
        nZero+=1
        nhitZero.append(nhit[iEvt])

    if(MuonHits[1,0] or MuonHits[1,2]):
        nSingle+=1
        nhitSingle.append(nhit[iEvt])
        if(MuonHits[1,1] or MuonHits[0,2]):
            #print("Double!",iEvt)
            nDouble+=1
            nhitDouble.append(nhit[iEvt])
            if(MuonHits[0,0] or MuonHits[0,1]):
                #print("Triple!",iEvt)
                nTriple+=1
                nhitTriple.append(nhit[iEvt])

print("Total Events=",len(event_number)," Zeros= ",nZero," Singles= ",nSingle," Doubles= ",nDouble, " Triples=",nTriple)
plt.figure(figsize=(10,6))

weightsZero = np.ones_like(nhitZero) / len(nhitZero)
weightsSingle = np.ones_like(nhitSingle) / len(nhitSingle)
weightsDouble = np.ones_like(nhitDouble) / len(nhitDouble)
weightsTriple = np.ones_like(nhitTriple) / len(nhitTriple)

plt.hist([nhitZero,nhitSingle,nhitDouble,nhitTriple],weights=[weightsZero,weightsSingle,weightsDouble,weightsTriple],label=["ZeroStacked","SingleStacked","DoubleStacked","TripleStacked"],
         color=["green","red","blue","black"],bins=range(0,225), stacked=True)
#plt.hist(nhitZero,color="green",alpha=0.7,label="ZeroNOTStacked",bins=range(0,225),density=True)

plt.xlabel("nhits",fontsize=18)
plt.ylabel("Each Histogram Normalized to 1",fontsize=15)
plt.legend()
#plt.yscale('log')
plt.show()

### Plotting Delta T for events with and without muon hits

In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

run_number = "312" #
#run_number = "408" #6 hour run with MTCA@120mV prescale of 10

rdf = ROOT.RDataFrame("events","/mnt/data/eos/run"+run_number+"/run"+run_number+".root")
nhit = rdf.AsNumpy(columns=["nhit"])["nhit"]
event_number = rdf.AsNumpy(columns=["event_number"])["event_number"]
board = rdf.AsNumpy(columns=["board"])["board"]
channel = rdf.AsNumpy(columns=["channel"])["channel"]
charge = rdf.AsNumpy(columns=["charge"])["charge"]
deltat = rdf.AsNumpy(columns=["deltat"])["deltat"]

nZero=0
nSingle=0
nDouble=0
nTriple=0

deltatZero = []
deltatSingle = []
deltatDouble = []
deltatTriple = []


for iEvt in range(0,len(event_number)-1):
    MuonHits = np.zeros([2,3])    
    for iHit in range(0,len(board[iEvt])):
        if  (board[iEvt][iHit] == 14 and channel[iEvt][iHit] == 12):
           MuonHits[0,0]=1
        elif(board[iEvt][iHit] == 14 and channel[iEvt][iHit] == 13):
           MuonHits[0,1]=1      
        elif(board[iEvt][iHit] == 14 and channel[iEvt][iHit] == 14):
           MuonHits[0,2]=1
        elif(board[iEvt][iHit] == 15 and channel[iEvt][iHit] == 12):
           MuonHits[1,0]=1
        elif(board[iEvt][iHit] == 15 and channel[iEvt][iHit] == 13):
           MuonHits[1,1]=1
        elif(board[iEvt][iHit] == 15 and channel[iEvt][iHit] == 14):
           MuonHits[1,2]=1

    if(np.sum(MuonHits)==0): 
        nZero+=1
        deltatZero.append(deltat[iEvt+1])

    if(MuonHits[1,0] or MuonHits[1,2]):
        nSingle+=1
        deltatSingle.append(deltat[iEvt+1])
        if(MuonHits[1,1] or MuonHits[0,2]):
            #print("Double!",iEvt)
            nDouble+=1
            deltatDouble.append(deltat[iEvt+1])
            if(MuonHits[0,0] or MuonHits[0,1]):
                #print("Triple!",iEvt)
                nTriple+=1
                deltatTriple.append(deltat[iEvt+1])

print("Total Events=",len(event_number)," Zeros= ",nZero," Singles= ",nSingle," Doubles= ",nDouble, " Triples=",nTriple)
plt.figure(figsize=(10,6))

weightsZero = np.ones_like(deltatZero) / len(deltatZero)
weightsSingle = np.ones_like(deltatSingle) / len(deltatSingle)
weightsDouble = np.ones_like(deltatDouble) / len(deltatDouble)
weightsTriple = np.ones_like(deltatTriple) / len(deltatTriple)

plt.hist(deltatZero,weights=weightsZero,label="Zero",color="green",bins=range(0,2000,10),alpha=0.5)
plt.hist(deltatDouble,weights=weightsDouble,label="Double",color="blue",bins=range(0,2000,10),alpha=0.5)
         
#plt.hist(nhitZero,color="green",alpha=0.7,label="ZeroNOTStacked",bins=range(0,225),density=True)

plt.title("$\Delta$t [$\mu$s] for Run "+run_number)
plt.xlabel("$\Delta$t [$\mu$s]",fontsize=18)
plt.ylabel("Each Histogram Normalized to 1",fontsize=15)
plt.legend()
plt.yscale('log')
plt.show()

In [ ]:
count, bin_edges = np.histogram(deltatZero, bins=np.arange(1,10,0.2))
bin_centers =[]
for ie in range(1,len(bin_edges)):
    bin_centers.append((bin_edges[ie]+bin_edges[ie-1])/2)

from scipy.optimize import curve_fit
# Create the fit as a function
# x = dependent var
# a b c are the parameters
def myfit(x,a,b,tau):
    return a+b*np.exp(x/tau)

param0 = (0,100,-2)
paramfit, paramErr = curve_fit(myfit,bin_centers,#[5:15],
                               count,#[5:15],
                               param0)

print(paramfit)
#paramfit[2]=2.2
fitfun = myfit(bin_centers,paramfit[0],paramfit[1],paramfit[2])


plt.figure(figsize=(10,8))
plt.title("$\Delta t$ histogram for Run "+run_number)
plt.xlabel("$\Delta t [\mu s]$")
plt.ylabel("Events Per Bin")
#plt.xlim(0,10)
plt.yscale('log')
plt.plot(bin_centers,fitfun,"r",label="A+B*exp(t/tau)")
plt.bar(bin_centers, count, color ='blue', width = 0.2, label="data")
plt.xlabel("$\Delta$t [$\mu$s]",fontsize=18)
plt.ylabel("Each Histogram Normalized to 1",fontsize=15)
plt.legend()
#plt.yscale('log')
plt.show()

### Let's cut on delta T < 10 and see what the nhit / charge histogram looks like.

In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

run_number = "312" #
#run_number = "408" #6 hour run with MTCA@120mV prescale of 10

rdf = ROOT.RDataFrame("events","/mnt/data/eos/run"+run_number+"/run"+run_number+".root")
nhit = rdf.AsNumpy(columns=["nhit"])["nhit"]
event_number = rdf.AsNumpy(columns=["event_number"])["event_number"]
board = rdf.AsNumpy(columns=["board"])["board"]
channel = rdf.AsNumpy(columns=["channel"])["channel"]
charge = rdf.AsNumpy(columns=["charge"])["charge"]
deltat = rdf.AsNumpy(columns=["deltat"])["deltat"]

nhit_lowDT = []


for iEvt in range(0,len(event_number)):
    if(deltat[iEvt] < 10):
        nhit_lowDT.append(nhit[iEvt]) 


count, bin_edges = np.histogram(nhit_lowDT, bins=np.arange(0,225,1))
bin_centers =[]
for ie in range(1,len(bin_edges)):
    bin_centers.append((bin_edges[ie]+bin_edges[ie-1])/2)


plt.bar(bin_centers, count, color ='blue', width = 1, label="data")

plt.title("nhit histogram for Run "+run_number)
plt.xlabel("nhit",fontsize=18)
plt.ylabel("NEvents",fontsize=15)

plt.show()

In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

run_number = "312" #
#run_number = "408" #6 hour run with MTCA@120mV prescale of 10

rdf = ROOT.RDataFrame("events","/mnt/data/eos/run"+run_number+"/run"+run_number+".root")
nhit = rdf.AsNumpy(columns=["nhit"])["nhit"]
event_number = rdf.AsNumpy(columns=["event_number"])["event_number"]
board = rdf.AsNumpy(columns=["board"])["board"]
channel = rdf.AsNumpy(columns=["channel"])["channel"]
total_charge = rdf.AsNumpy(columns=["total_charge"])["total_charge"]
deltat = rdf.AsNumpy(columns=["deltat"])["deltat"]

nhit_lowDT = []
total_charge_lowDT =[]


for iEvt in range(0,len(event_number)):
    if(deltat[iEvt] < 10):
        nhit_lowDT.append(nhit[iEvt])
        total_charge_lowDT.append(total_charge[iEvt])


count, bin_edges = np.histogram(total_charge_lowDT, bins=np.arange(0,2000,20))
bin_centers =[]
for ie in range(1,len(bin_edges)):
    bin_centers.append((bin_edges[ie]+bin_edges[ie-1])/2)


plt.bar(bin_centers, count, color ='blue', width = 20, label="data")

plt.title("total_charge histogram for Run "+run_number)
plt.xlabel("total_charge",fontsize=18)
plt.ylabel("NEvents",fontsize=15)

plt.show()

In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

run_number = "357"

rdf = ROOT.RDataFrame("events","/mnt/data/eos/run"+run_number+"/run"+run_number+".root")
nhit = rdf.AsNumpy(columns=["nhit"])["nhit"]
event_number = rdf.AsNumpy(columns=["event_number"])["event_number"]
board = rdf.AsNumpy(columns=["board"])["board"]
channel = rdf.AsNumpy(columns=["channel"])["channel"]
charge = rdf.AsNumpy(columns=["charge"])["charge"]
deltat = rdf.AsNumpy(columns=["deltat"])["deltat"]

nZero=0
nSingle=0
nDouble=0
nTriple=0

deltatZero = []
deltatSingle = []
deltatDouble = []
deltatTriple = []


for iEvt in range(0,len(event_number)):
    MuonHits = np.zeros([2,3])    
    for iHit in range(0,len(board[iEvt])):
        if  (board[iEvt][iHit] == 14 and channel[iEvt][iHit] == 12):
           MuonHits[0,0]=1
        elif(board[iEvt][iHit] == 14 and channel[iEvt][iHit] == 13):
           MuonHits[0,1]=1      
        elif(board[iEvt][iHit] == 14 and channel[iEvt][iHit] == 14):
           MuonHits[0,2]=1
        elif(board[iEvt][iHit] == 15 and channel[iEvt][iHit] == 12):
           MuonHits[1,0]=1
        elif(board[iEvt][iHit] == 15 and channel[iEvt][iHit] == 13):
           MuonHits[1,1]=1
        elif(board[iEvt][iHit] == 15 and channel[iEvt][iHit] == 14):
           MuonHits[1,2]=1

    if(np.sum(MuonHits)==0): 
        nZero+=1
        deltatZero.append(deltat[iEvt])

    if(MuonHits[1,0] or MuonHits[1,2]):
        nSingle+=1
        deltatSingle.append(deltat[iEvt])
        if(MuonHits[1,1] or MuonHits[0,2]):
            #print("Double!",iEvt)
            nDouble+=1
            deltatDouble.append(deltat[iEvt])
            if(MuonHits[0,0] or MuonHits[0,1]):
                #print("Triple!",iEvt)
                nTriple+=1
                deltatTriple.append(deltat[iEvt])

print("Total Events=",len(event_number)," Zeros= ",nZero," Singles= ",nSingle," Doubles= ",nDouble, " Triples=",nTriple)
plt.figure(figsize=(10,6))

weightsZero = np.ones_like(deltatZero) / len(deltatZero)
weightsSingle = np.ones_like(deltatSingle) / len(deltatSingle)
weightsDouble = np.ones_like(deltatDouble) / len(deltatDouble)
weightsTriple = np.ones_like(deltatTriple) / len(deltatTriple)

plt.hist(deltatZero,weights=weightsZero,label="Zero",color="green",bins=range(200,24000,200),alpha=0.5)
plt.hist(deltatDouble,weights=weightsDouble,label="Double",color="blue",bins=range(200,24000,200),alpha=0.5)
         
#plt.hist(nhitZero,color="green",alpha=0.7,label="ZeroNOTStacked",bins=range(0,225),density=True)

plt.xlabel("$\Delta$t [$\mu$s]",fontsize=18)
plt.ylabel("Each Histogram Normalized to 1",fontsize=15)
plt.legend()
plt.yscale('log')
plt.show()

# MC Analysis Below

In [ ]:
##################### MC Analysis Below this.
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
f = ROOT.TFile.Open("/mnt/data/eos/simulations/muon/eos_muon_water_trigthres30_70kcomb.root")
output = f.Get("output")
output.Print()

## This block loops over the SIM to see if there's a muon neutrino in the event.
## These events are assumed to be muon decays.

In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
rdf = ROOT.RDataFrame("output","/mnt/data/eos/simulations/muon/eos_muon_water_trigthres30_70kcomb.root")
mcpdg = rdf.AsNumpy(columns=["mcpdg"])["mcpdg"]
trackPDG = rdf.AsNumpy(columns=["trackPDG"])["trackPDG"]
nhits = rdf.AsNumpy(columns=["nhits"])["nhits"]

nhitsWithMuNu=[]

nEventWithMuNu=0
for iEvt in range(0,len(trackPDG)):
    if(iEvt%100 == 0): print("Event#:",iEvt)
    
    FoundMuNu = False
    for iTrk in range(0,len(trackPDG[iEvt])):
        
        if(trackPDG[iEvt][iTrk] == 14 or trackPDG[iEvt][iTrk] == -14):
            #print(trackPDG[iEvt][iTrk])
            FoundMuNu = True

    if(FoundMuNu): 
        nEventWithMuNu+=1
        nhitsWithMuNu.append(nhits[iEvt])

plt.hist(nhitsWithMuNu,bins=range(0,225,1))
print("nEventWithMuNu/nEvent ",nEventWithMuNu/len(trackPDG)) 


Event#: 0
Event#: 100
Event#: 200
Event#: 300
Event#: 400
Event#: 500
Event#: 600
Event#: 700
Event#: 800
Event#: 900
Event#: 1000
Event#: 1100
Event#: 1200
Event#: 1300
Event#: 1400
Event#: 1500
Event#: 1600
Event#: 1700
Event#: 1800
Event#: 1900
Event#: 2000
Event#: 2100
Event#: 2200
Event#: 2300
Event#: 2400
Event#: 2500
Event#: 2600
Event#: 2700
Event#: 2800
Event#: 2900
Event#: 3000
Event#: 3100
Event#: 3200
Event#: 3300
Event#: 3400
Event#: 3500
Event#: 3600
Event#: 3700
Event#: 3800
Event#: 3900
Event#: 4000
Event#: 4100
Event#: 4200
Event#: 4300
Event#: 4400
Event#: 4500


## This plots the nhits of the muon SIMS.

In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

rdf = ROOT.RDataFrame("output","/mnt/data/eos/simulations/muon/eos_muon_water_trigthres30_70kcomb.root")
nhits = rdf.AsNumpy(columns=["nhits"])["nhits"]
plt.hist(nhits,bins=range(0,225))
plt.show()
print(len(nhits))

## This plots the triggerTime of the muon SIMS.

In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

rdf = ROOT.RDataFrame("output","/mnt/data/eos/simulations/muon/eos_muon_water_trigthres30_70kcomb.root")
triggerTime = rdf.AsNumpy(columns=["triggerTime"])["triggerTime"]
#for iEvt in range(0,100):#len(triggerTime)):
    #for iTrig in range(0,len(
#    print(triggerTime[iEvt])
plt.hist(triggerTime,bins=range(1,35,1))
plt.yscale('log')
plt.show()
